In [0]:
df_conversations_bronze_history = spark.sql("DESCRIBE HISTORY slack_etl.bronze.conversations LIMIT 1")
conversations_bronze_latest_commit_version = df_conversations_bronze_history.orderBy('version', ascending=False).limit(1).collect()[0].asDict()['version']

In [0]:
df_conversations_bronze = spark.table('slack_etl.bronze.conversations')


In [0]:

df_conversations_silver_enriched = df_conversations_bronze.selectExpr("created","id","members", "name AS channel_name","purpose.creator AS creator","purpose.value AS purpose","topic.value AS topic")

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

w = Window.partitionBy('id').orderBy('created')
df_conversations_silver_dedup = df_conversations_silver_enriched.withColumn('row_num', row_number().over(w)).filter('row_num = 1').drop('row_num')

In [0]:
df_conversations_silver_dedup.write.mode('overwrite').saveAsTable('slack_etl.silver.conversations')